# Feature performance comparison

The purpose of this notebook is to compare the classification performance of the individual features, and their combination, for E Coli

# Imports

In [1]:
import os
import sys
from IPython.display import display

sys.path.append('../src')
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA, KernelPCA
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.feature_selection import SelectKBest, RFE, VarianceThreshold
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.base import clone
from scipy.stats import shapiro
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator,TransformerMixin

from yellowbrick.features import ParallelCoordinates
from yellowbrick.features import Rank1D, Rank2D

import pandas as pd
import numpy as np
import seaborn as sns

from dataset.transporter_dataset import create_dataset
from dataset.cluster_fasta import cd_hit
from features.labels import fasta_to_labels
from features.compositions import calculate_composition_feature
from features.pssm import calculate_pssm_feature
from features.coexp import calculate_coexp_feature
from models.eval import nested_crossval
from visualization.feature_plots import create_plot

# Globals

In [2]:
N_THREADS = 16
IDENTITY_THRESHOLD=70
TAX_ID = 9606
ORGANISM = "human"
LOG_FILE = "../logs/human_amino_sugar.log"

# Dataset

In [3]:
# Delete previous log
if os.path.exists(LOG_FILE):
    with open(LOG_FILE, "w"):
        pass

create_dataset(
    keywords_substrate_filter=["Amino-acid transport", "Sugar transport"],
    keywords_component_filter=["Membrane"],
    keywords_transport_filter=["Transport"],
    input_file="../data/raw/swissprot/uniprot-reviewed_yes.tab.gz",
    multi_substrate="integrate",
    verbose=True,
    outliers=["Q9HBR0", "Q07837"],
    tax_ids_filter=[TAX_ID],
    output_tsv=f"../data/datasets/{ORGANISM}_amino_sugar.tsv",
    output_fasta=f"../data/datasets/{ORGANISM}_amino_sugar.fasta",
    output_log=LOG_FILE,
)


## Clustering

In [4]:
cd_hit(
    executable_location="cd-hit",
    input_fasta=f"../data/datasets/{ORGANISM}_amino_sugar.fasta",
    output_fasta=f"../data/datasets/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}.fasta",
    log_file=LOG_FILE,
    identity_threshold=IDENTITY_THRESHOLD,
    n_threads=N_THREADS,
    memory=4096,
    verbose=True,
)

## Annotations

In [5]:
df_annotations = pd.read_table(f"../data/datasets/{ORGANISM}_amino_sugar.tsv", index_col=0)
df_annotations.head()

,keywords_transport,keywords_location,keywords_transport_related,gene_names,protein_names,tcdb_id,organism_id,sequence
Uniprot,,,,,,,,
Q9BWM7,Amino-acid transport,Membrane;Mitochondrion;Transmembrane,Transport,SFXN3,Sideroflexin-3,NaN,9606,MGELPLDINIQEPRWDQSTFLGRARHFFTVTDPRNLLLSGAQLEAS...
Q9BRV3,Sugar transport,Cell membrane;Membrane;Transmembrane,Transport,SLC50A1 RAG1AP1 SCP,Sugar transporter SWEET1 (HsSWEET1) (RAG1-acti...,2.A.123.1.4,9606,MEAGGFLDSLIYGACVVFTLGMFSAGLSDLRHMRMTRSVDNVQFLP...
Q5M8T2,Sugar transport,Membrane;Transmembrane,Transport,SLC35D3 FRCL1,Solute carrier family 35 member D3 (Fringe con...,2.A.7.15.5,9606,MRQLCRGRVLGISVAIAHGVFSGSLNILLKFLISRYQFSFLTLVQC...
Q969S0,Sugar transport,Membrane;Transmembrane,Transport,SLC35B4 YEA4 PSEC0055,UDP-xylose and UDP-N-acetylglucosamine transpo...,2.A.7.10.2,9606,MRPALAVGLVFAGCCSNVIFLELLARKHPGCGNIVTFAQFLFIAVE...
O75387,Amino-acid transport,Membrane;Transmembrane,Transport,SLC43A1 LAT3 PB39 POV1,Large neutral amino acids transporter small su...,2.A.1.44.1,9606,MAPTLQQAYRRRWWMACTAVLENLFFSAVLLGWGSLLIILKNEGFY...


# Feature generation

## Labels

In [6]:
fasta_to_labels(
    input_fasta=f"../data/datasets/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}.fasta",
    output_tsv=f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_labels.tsv",
)
df_labels = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_labels.tsv",
    index_col=0,
)
df_labels.labels.value_counts()

Amino-acid transport    48
Sugar transport         34
Name: labels, dtype: int64

## AAC, PAAC

In [7]:
for composition_type in ["aac", "paac"]:
    calculate_composition_feature(
        input_fasta=f"../data/datasets/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}.fasta",
        output_tsv=f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_{composition_type}.tsv",
        feature_type=composition_type,
    )

## PSSM

In [8]:
for uniref_cluster_threshold in [50, 90]:
    for psiblast_iterations in [1, 3]:
        calculate_pssm_feature(
            input_fasta=f"../data/datasets/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}.fasta",
            output_tsv="../data/features/{}_amino_sugar_cluster{}_pssm_ur{}_{}it.tsv".format(
                ORGANISM, IDENTITY_THRESHOLD, uniref_cluster_threshold, psiblast_iterations
            ),
            tmp_folder="../data/intermediate/blast/pssm_uniref{}_{}it".format(
                uniref_cluster_threshold, psiblast_iterations
            ),
            blast_db="../data/raw/uniref/uniref{}/uniref{}.fasta".format(
                uniref_cluster_threshold, uniref_cluster_threshold
            ),
            iterations=psiblast_iterations,
            psiblast_executable="psiblast",
            psiblast_threads=N_THREADS,
            verbose=False,
        )

## COEXP

In [9]:
# TODO: optimize parameters first

## Reading dataframes

In [10]:

df_aac = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_aac.tsv",
    index_col=0,
)
df_paac = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_paac.tsv",
    index_col=0,
)
df_pssm_50_1it = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_pssm_ur50_1it.tsv",
    index_col=0,
)
df_pssm_50_3it = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_pssm_ur50_3it.tsv",
    index_col=0,
)
df_pssm_90_1it = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_pssm_ur90_1it.tsv",
    index_col=0,
)
df_pssm_90_3it = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_pssm_ur90_3it.tsv",
    index_col=0,
)

## Combining dataframes

In [11]:
df_all = pd.concat(
    [
        df_aac,
        df_paac,
        df_pssm_50_1it.rename(columns=lambda c: "PSSM_" + c + "_50_1"),
        df_pssm_50_3it.rename(columns=lambda c: "PSSM_" + c + "_50_3"),
        df_pssm_90_1it.rename(columns=lambda c: "PSSM_" + c + "_90_1"),
        df_pssm_90_3it.rename(columns=lambda c: "PSSM_" + c + "_90_3"),
    ],
    axis=1,
)

df_all

,A,C,D,E,F,G,H,I,K,L,...,PSSM_VL_90_3,PSSM_VK_90_3,PSSM_VM_90_3,PSSM_VF_90_3,PSSM_VP_90_3,PSSM_VS_90_3,PSSM_VT_90_3,PSSM_VW_90_3,PSSM_VY_90_3,PSSM_VV_90_3
Uniprot,,,,,,,,,,,,,,,,,,,,,
Q9BRV3,0.058824,0.022624,0.031674,0.018100,0.058824,0.072398,0.009050,0.045249,0.022624,0.176471,...,0.484375,0.403125,0.471875,0.706250,0.368750,0.443750,0.440625,0.568750,0.856250,0.478125
Q5M8T2,0.096154,0.024038,0.024038,0.050481,0.038462,0.093750,0.012019,0.052885,0.026442,0.129808,...,0.522388,0.492537,0.525373,0.659701,0.498507,0.516418,0.513433,0.594030,0.817910,0.498507
Q969S0,0.069486,0.018127,0.015106,0.033233,0.090634,0.060423,0.018127,0.090634,0.045317,0.126888,...,0.492669,0.398827,0.492669,0.568915,0.384164,0.422287,0.416422,0.609971,0.750733,0.472141
O75387,0.076923,0.026834,0.028623,0.035778,0.064401,0.078712,0.012522,0.046512,0.028623,0.148479,...,0.527546,0.510851,0.509182,0.602671,0.477462,0.514190,0.515860,0.544240,0.686144,0.512521
Q9NTN3,0.090141,0.011268,0.030986,0.039437,0.076056,0.076056,0.005634,0.061972,0.061972,0.129577,...,0.450704,0.357746,0.439437,0.645070,0.321127,0.388732,0.397183,0.594366,0.867606,0.450704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q96A29,0.082418,0.024725,0.024725,0.032967,0.054945,0.079670,0.008242,0.038462,0.038462,0.162088,...,0.547126,0.457471,0.531034,0.655172,0.427586,0.487356,0.482759,0.634483,0.749425,0.533333
Q9BYW1,0.098790,0.032258,0.012097,0.054435,0.048387,0.100806,0.010081,0.064516,0.018145,0.157258,...,0.534107,0.502177,0.528302,0.647315,0.489115,0.516691,0.513788,0.579100,0.718433,0.523948
P14672,0.086444,0.005894,0.011788,0.051081,0.053045,0.104126,0.007859,0.066798,0.015717,0.155206,...,0.514116,0.476969,0.505201,0.618128,0.471025,0.490342,0.484398,0.551263,0.716196,0.494799


## Custom Transformer to try all parameters

New version: Ignores all additional features that don't start with "PSSM"

In [12]:
class PSSMSelector(BaseEstimator, TransformerMixin):
    def __init__(self, feature_names, uniref_threshold="all", iterations="all"):
        self.feature_names = feature_names
        self.uniref_threshold = uniref_threshold
        self.iterations = iterations

    def fit(self, X, y=None):
        if self.uniref_threshold in {50, 90}:
            has_uniref = (
                np.char.find(self.feature_names, str(self.uniref_threshold)) >= 0
            )
        elif self.uniref_threshold == "all":
            has_uniref = np.array([True] * len(self.feature_names))
        else:
            raise ValueError(f"Incorrect uniref threshold {self.uniref_threshold}")

        if self.iterations in {1, 3}:
            has_iterations = np.char.find(self.feature_names, str(self.iterations)) >= 0
        elif self.iterations == "all":
            has_iterations = np.array([True] * len(self.feature_names))
        else:
            raise ValueError(f"Incorrect iteration count: {self.iterations}")
        
        not_pssm_feature = ~np.char.startswith(self.feature_names, "PSSM")
        self.mask = np.bitwise_or(np.bitwise_and(has_uniref, has_iterations), not_pssm_feature)
        return self

    def transform(self, X, y=None):
        X = np.array(X)
        X = X[:, self.mask]
        return X


# Functions

In [13]:
def get_feature_stats(df_features, df_labels_, labels=["Amino-acid transport", "Sugar transport"]):
    df_stats = pd.concat(
        {
            "corr": df_features.corrwith(
                df_labels_.labels.transform(lambda x: 1.0 if x == labels[1] else 0.0)
            ),
            "mean": df_features.mean(),
            "std": df_features.std(),
        },
        axis=1,
    )

    df_stats["sum"] = df_stats.sum(axis=1)
    df_stats["corr_abs"] = df_stats["corr"].abs()

    df_stats["mean0"] = df_features.loc[df_labels_[df_labels_.labels == labels[0]].index].mean()
    df_stats["mean1"] = df_features.loc[df_labels_[df_labels_.labels == labels[1]].index].mean()

    df_stats["median0"] = df_features.loc[
        df_labels_[df_labels_.labels == labels[0]].index
    ].median()
    df_stats["median1"] = df_features.loc[
        df_labels_[df_labels_.labels == labels[1]].index
    ].median()

    df_stats["mediandiff"] = (df_stats["median0"] - df_stats["median1"]).abs()
    df_stats = df_stats.sort_values("mediandiff", ascending=False)
    return df_stats

In [14]:
def get_independent_test_set(
    df_features, df_labels_, labels=["Amino-acid transport", "Sugar transport"], test_size=0.2
):
    X = df_features.to_numpy()
    y = np.where(df_labels_.labels == labels[1], 1, 0)
    feature_names = df_features.columns.to_numpy(dtype=str)
    sample_names = df_features.index.to_numpy(dtype=str)
    (
        X_train,
        X_test,
        y_train,
        y_test,
        sample_names_train,
        sample_names_test,
    ) = train_test_split(
        X, y, sample_names, stratify=y, random_state=42, shuffle=True, test_size=test_size
    )
    return (
        X_train,
        X_test,
        y_train,
        y_test,
        sample_names_train,
        sample_names_test,
        feature_names,
    )


In [15]:
def print_validation_results(y_true_, y_pred_, labels = ["Amino", "Sugar"]):
    report_dict = classification_report(y_true=y_true_, y_pred=y_pred_, output_dict=True)
    report_dict = {
        labels[0]: report_dict['0'],
        labels[1]: report_dict['1'],
        "Macro": report_dict["macro avg"],
        "Weighted": report_dict["weighted avg"]
    }
    report_df = pd.DataFrame.from_dict(report_dict)
    confusion_matrix_df = pd.DataFrame(
        confusion_matrix(y_true_, y_pred_),
        columns=labels,
        index=labels,
    )
    return report_df, confusion_matrix_df

# Combined Features

### Stats, Plots

Only three of the top 30 features comes from PAAC, all others come from PSSM with Uniref50 and 3 iterations. This is also the best parameter PSSM dataset that was found in the PSSM notebook.

In [16]:
df_stats = get_feature_stats(df_all, df_labels)
df_stats.sort_values("corr_abs", ascending=False).head(30)

,corr,mean,std,sum,corr_abs,mean0,mean1,median0,median1,mediandiff
PSSM_MP_50_1,-0.485078,0.858349,0.087182,0.460453,0.485078,0.893724,0.808408,0.878336,0.803954,0.074382
PSSM_MP_50_3,-0.461428,0.850042,0.073508,0.462122,0.461428,0.878415,0.809988,0.867102,0.811779,0.055323
PSSM_MQ_90_3,0.450707,0.451932,0.060241,0.962880,0.450707,0.429221,0.483995,0.436009,0.488064,0.052056
PSSM_MM_50_3,0.441090,0.497208,0.078699,1.016996,0.441090,0.468171,0.538201,0.458843,0.539064,0.080221
PSSM_MQ_50_3,0.431143,0.511968,0.065452,1.008562,0.431143,0.488363,0.545292,0.484587,0.542313,0.057726
PSSM_MK_90_3,0.416819,0.452357,0.057272,0.926448,0.416819,0.432388,0.480548,0.437552,0.484883,0.047331
PSSM_MN_90_3,0.410304,0.430152,0.065035,0.905491,0.410304,0.407832,0.461664,0.414027,0.475172,0.061144
PSSM_MR_50_3,0.409897,0.491311,0.072910,0.974118,0.409897,0.466312,0.526603,0.463320,0.528278,0.064958
PSSM_MK_50_3,0.405128,0.504827,0.070182,0.980137,0.405128,0.481043,0.538403,0.471594,0.538712,0.067118
PSSM_MI_50_3,0.403964,0.491396,0.080309,0.975670,0.403964,0.464259,0.529707,0.464983,0.532613,0.067630


### Independent test set

In [17]:
(
    X_train,
    X_test,
    y_train,
    y_test,
    sample_names_train,
    sample_names_test,
    feature_names,
) = get_independent_test_set(df_all, df_labels, test_size=0.2)

### Model selection

Linear SVC could be a good choice to avoid overfitting.

In [18]:
for estimator in [
    LinearSVC(max_iter=1e6, class_weight="balanced"),
    SVC(class_weight="balanced"),
    RandomForestClassifier(class_weight="balanced"),
    LinearSVC(max_iter=1e6),
    SVC(),
    RandomForestClassifier(),
    GaussianNB(),
    KNeighborsClassifier(),
    SGDClassifier(),
]:
    pipe = make_pipeline(StandardScaler(), estimator)
    scores = cross_val_score(pipe, X_train, y_train, scoring="f1_macro")
    print("### ", str(estimator))
    print(f"CV folds: {scores.round(3)}")
    print(f"Mean: {scores.mean().round(3)}")
    print(f"Std: {scores.std().round(3)}")


###  LinearSVC(class_weight='balanced', max_iter=1000000.0)
CV folds: [0.764 0.845 0.845 0.675 0.511]
Mean: 0.728
Std: 0.125
###  SVC(class_weight='balanced')
CV folds: [0.675 0.923 0.764 0.764 0.458]
Mean: 0.717
Std: 0.152
###  RandomForestClassifier(class_weight='balanced')
CV folds: [0.675 0.838 0.675 0.639 0.458]
Mean: 0.657
Std: 0.121
###  LinearSVC(max_iter=1000000.0)
CV folds: [0.764 0.845 0.845 0.675 0.511]
Mean: 0.728
Std: 0.125
###  SVC()
CV folds: [0.675 0.745 0.838 0.764 0.458]
Mean: 0.696
Std: 0.13
###  RandomForestClassifier()
CV folds: [0.675 0.745 0.764 0.575 0.639]
Mean: 0.68
Std: 0.069
###  GaussianNB()
CV folds: [0.69  0.764 0.69  0.764 0.575]
Mean: 0.697
Std: 0.069
###  KNeighborsClassifier()
CV folds: [0.764 0.845 0.675 0.69  0.513]
Mean: 0.697
Std: 0.11
###  SGDClassifier()
CV folds: [0.769 0.923 0.921 0.606 0.639]
Mean: 0.772
Std: 0.134


### Parameter tuning

In [19]:
gsearch = GridSearchCV(
    estimator=make_pipeline(
        PSSMSelector(feature_names=feature_names),
        StandardScaler(),
        LinearSVC(max_iter=1e6),
    ),
    param_grid={
        "pssmselector__uniref_threshold": [50, 90, "all"],
        "pssmselector__iterations": [1, 3, "all"],
        "linearsvc__class_weight": ["balanced", None],
        "linearsvc__C": [1, 10, 100],
        "linearsvc__dual": [True, False],
    },
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,
    return_train_score=True,
)
gsearch.fit(X_train, y_train)
print(gsearch.best_params_)
print(gsearch.best_score_)
best_estimator_lsvc = gsearch.best_estimator_


{'linearsvc__C': 10, 'linearsvc__class_weight': 'balanced', 'linearsvc__dual': False, 'pssmselector__iterations': 1, 'pssmselector__uniref_threshold': 50}
0.8067302305537598


In [20]:
gsearch = GridSearchCV(
    estimator=make_pipeline(
        PSSMSelector(feature_names=feature_names),
        StandardScaler(),
        SVC(max_iter=1e6),
    ),
    param_grid={
        "pssmselector__uniref_threshold": [50, 90, "all"],
        "pssmselector__iterations": [1, 3, "all"],
        "svc__class_weight": ["balanced", None],
        "svc__C": [1, 10, 100],
        "svc__gamma": ["scale", 0.1, 0.01, 0.001],
    },
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,
    return_train_score=True,
)
gsearch.fit(X_train, y_train)
print(gsearch.best_params_)
print(gsearch.best_score_)
best_estimator_svc = gsearch.best_estimator_


{'pssmselector__iterations': 1, 'pssmselector__uniref_threshold': 'all', 'svc__C': 10, 'svc__class_weight': 'balanced', 'svc__gamma': 'scale'}
0.8244846965899597


### Dimensionality reduction

In [21]:
pca = PCA()
pca.fit(X_train)
csum = np.cumsum(pca.explained_variance_ratio_)
print("Number of components to explain 97% of variance:", np.argmax(csum >= 0.97) + 1)

Number of components to explain 97% of variance: 33


In [24]:
gsearch = GridSearchCV(
    estimator=make_pipeline(
        PSSMSelector(feature_names=feature_names),
        StandardScaler(),
        PCA(),
        StandardScaler(),
        LinearSVC(max_iter=1e6),
    ),
    param_grid={
        "pssmselector__uniref_threshold": [50, 90, "all"],
        "pssmselector__iterations": [1, 3, "all"],
        "linearsvc__class_weight": ["balanced", None],
        "linearsvc__C": [1, 0.001, 0.01, 0.1],
        "linearsvc__dual": [True, False],
        "pca__n_components": np.linspace(0.8, 0.99, 20)
    },
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,
    return_train_score=True,
)
gsearch.fit(X_train, y_train)
print(gsearch.best_params_)
print(gsearch.best_score_)
best_estimator_lsvc_pca = gsearch.best_estimator_

{'linearsvc__C': 0.001, 'linearsvc__class_weight': 'balanced', 'linearsvc__dual': True, 'pca__n_components': 0.98, 'pssmselector__iterations': 1, 'pssmselector__uniref_threshold': 50}
0.8067302305537598


In [25]:
gsearch = GridSearchCV(
    estimator=make_pipeline(
        PSSMSelector(feature_names=feature_names),
        StandardScaler(),
        PCA(),
        StandardScaler(),
        SVC(max_iter=1e6),
    ),
    param_grid={
        "pssmselector__uniref_threshold": [50, 90, "all"],
        "pssmselector__iterations": [1, 3, "all"],
        "svc__class_weight": ["balanced", None],
        "svc__C": [0.1, 1, 10],
        "svc__gamma": ["scale", 0.01, 0.1, 1],
        "pca__n_components": np.linspace(0.8, 0.99, 20)
    },
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,
    return_train_score=True,
)
gsearch.fit(X_train, y_train)
print(gsearch.best_params_)
print(gsearch.best_score_)
best_estimator_svc_pca = gsearch.best_estimator_

{'pca__n_components': 0.87, 'pssmselector__iterations': 'all', 'pssmselector__uniref_threshold': 50, 'svc__C': 1, 'svc__class_weight': 'balanced', 'svc__gamma': 0.1}
0.8364213564213564


### Validation

In [26]:
best_estimator = best_estimator_svc_pca
best_scores = cross_val_score(
    estimator=clone(best_estimator), X=X_train, y=y_train, scoring="f1_macro"
)
print(f"Train scores: {best_scores.mean().round(3)}+-{best_scores.std().round(3)}")

y_pred = best_estimator.predict(X_test)
y_true = y_test.copy()

report_df, confusion_matrix_df = print_validation_results(y_true, y_pred, labels=["Amino", "Sugar"])
display(report_df.round(3))
display(confusion_matrix_df)

Train scores: 0.836+-0.09


,Amino,Sugar,Macro,Weighted
precision,0.833,1.000,0.917,0.902
recall,1.000,0.714,0.857,0.882
f1-score,0.909,0.833,0.871,0.878
support,10.000,7.000,17.000,17.000


,Amino,Sugar
Amino,10,0
Sugar,2,5


In [27]:
best_estimator = best_estimator_lsvc_pca
best_scores = cross_val_score(
    estimator=clone(best_estimator), X=X_train, y=y_train, scoring="f1_macro"
)
print(f"Train scores: {best_scores.mean().round(3)}+-{best_scores.std().round(3)}")

y_pred = best_estimator.predict(X_test)
y_true = y_test.copy()

report_df, confusion_matrix_df = print_validation_results(y_true, y_pred, labels=["Amino", "Sugar"])
display(report_df.round(3))
display(confusion_matrix_df)

Train scores: 0.807+-0.087


,Amino,Sugar,Macro,Weighted
precision,1.000,0.778,0.889,0.908
recall,0.800,1.000,0.900,0.882
f1-score,0.889,0.875,0.882,0.883
support,10.000,7.000,17.000,17.000


,Amino,Sugar
Amino,8,2
Sugar,0,7


# Alternative

In [28]:
from sklearn.model_selection import LeaveOneOut

X = df_all.values
labels = df_labels.labels
y = np.where(labels == "Sugar transport", 1, 0)


params_svc = {
    "svc__class_weight": ["balanced"],
    "svc__C": [0.1, 1, 10],
    # "svc__gamma": ["scale", 1e-1, 1e-2, 1e-3],
    "pssmselector__uniref_threshold": [50, 90, "all"],
    "pssmselector__iterations": [1, 3, "all"],
}
gsearch = GridSearchCV(
    make_pipeline(PSSMSelector(feature_names=feature_names), StandardScaler(), SVC()),
    param_grid=params_svc,
    cv=LeaveOneOut(),
    scoring="f1_macro",
    return_train_score=True,
    refit=True,
)
# the score does not make a difference, since it's either 1 (correct) or 0 (incorrect)
res = cross_val_predict(gsearch, X, y, n_jobs=-1, cv=LeaveOneOut())
print(f1_score(y, res, average="macro"))


params_svc = {
    "svc__class_weight": ["balanced"],
    "svc__C": [0.1, 1, 10],
    # "svc__gamma": ["scale", 1e-1, 1e-2, 1e-3],
    "pca__n_components": [0.8, 0.9],
    "pssmselector__uniref_threshold": [50, 90, "all"],
    "pssmselector__iterations": [1, 3, "all"],
}
gsearch = GridSearchCV(
    make_pipeline(
        PSSMSelector(feature_names=feature_names),
        StandardScaler(),
        PCA(),
        StandardScaler(),
        SVC(),
    ),
    param_grid=params_svc,
    cv=LeaveOneOut(),
    scoring="f1_macro",
    return_train_score=True,
    refit=True,
)
# the score does not make a difference, since it's either 1 (correct) or 0 (incorrect)
res = cross_val_predict(gsearch, X, y, n_jobs=-1, cv=LeaveOneOut())
print(f1_score(y, res, average="macro"))


0.8864440683182027
0.8853146853146854
